In [17]:
import torch
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall
import matplotlib.pyplot as plt
from torchvision import datasets
import torchvision.transforms as transforms

In [18]:
train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform = transforms.ToTensor())
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

In [19]:
classes = train_data.classes
num_classes = len(train_data.classes)
print(f"classes: {classes}\nnum_classes: {num_classes}")

classes: ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
num_classes: 10


In [20]:
num_input_channels = 1
num_output_channels = 16
image_size = train_data[0][0].shape[1]
image_size

28

In [21]:
class MultiClassImageClassifier(nn.Module):
  
    def __init__(self, num_classes):
        super(MultiClassImageClassifier, self).__init__()
        self.conv1 = nn.Conv2d(num_input_channels, num_output_channels, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

        # Create a fully connected layer
        self.fc = nn.Linear(num_output_channels * (image_size//2)**2, num_classes)
        
    def forward(self, x):
        # Pass inputs through each layer
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

In [22]:
dataloader_train = DataLoader(train_data, shuffle=True, batch_size = 10,)

In [23]:
def train_model(optimizer, net, num_epochs):
    num_processed = 0
    criterion = nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        running_loss = 0.0
        num_processed = 0
        for feature,label in dataloader_train:
            optimizer.zero_grad()
            outputs = net(feature)
            loss = criterion(outputs, label)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            num_processed += len(label)
        print(f"epoch:{epoch}, loss:{running_loss/num_processed}")
        
    train_loss = running_loss / len(dataloader_train)

In [24]:
net = MultiClassImageClassifier(num_classes)
optimizer = optim.Adam(net.parameters(), lr = 0.001)

NameError: name 'Net' is not defined

In [ ]:
train_model(
    optimizer=optimizer,
    net=net,
    num_epochs=1,
)

In [ ]:
dataloader_test = DataLoader(test_data,
                            shuffle = False,
                            batch_size =10,)

In [ ]:
# Define the metrics
accuracy_metric = Accuracy(task='multiclass', num_classes=num_classes)
precision_metric = Precision(task='multiclass', num_classes=num_classes, average=None)
recall_metric = Recall(task='multiclass', num_classes=num_classes, average=None)

# Run model on test set
net.eval()
predicted = []
for i, (features, labels) in enumerate(dataloader_test):
    output = net.forward(features.reshape(-1, 1, image_size, image_size))
    cat = torch.argmax(output, dim=-1)
    predicted.extend(cat.tolist())
    accuracy_metric(cat, labels)
    precision_metric(cat, labels)
    recall_metric(cat, labels)

# Compute the metrics
accuracy = accuracy_metric.compute().item()
precision = precision_metric.compute().tolist()
recall = recall_metric.compute().tolist()
print('Accuracy:', accuracy)
print('Precision (per class):', precision)
print('Recall (per class):', recall)